## Predicting Maximum Battery Capacity from Cycling Data

Noah H. Paulson, Saurabh Saxena

Applied Materials Division

Argonne National Laboratory

### Introduction

This is an jupyter notebook intended to teach some of the basics
of data analytics and data processing for the problem of predicting
the maximum capacity of a Li-ion battery as a function of its
voltage and current versus time signature for a particular cycle.

We will be using cycling data described in the following publication:
Severson, K.A., Attia, P.M., Jin, N. et al. Data-driven prediction of battery cycle life before capacity degradation. Nat Energy 4, 383–391 (2019). https://doi.org/10.1038/s41560-019-0356-8

#### Download the data file

In [ ]:
import gdown

url = 'https://drive.google.com/uc?export=download&id=1wcf5ctLG8TLnHN4lt3OJSpk5nmZcyY3z'
output = 'Standford_Batch1_CH1.h5'
gdown.download(url, output, quiet=False) 

#### Load the initial modules and the data file

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(0)

fname = 'Standford_Batch1_CH1.h5'
df = pd.read_hdf(fname)

# print out the column headers
print(df.keys())

# how many cycles are there?
ncycmax = df['Cycle Num'].max()
print('\nnumber of cycles:', ncycmax)

#### Let's plot some quantities we may be interested in
Start off with current over the life of the cell

In [ ]:
# import plotting module
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

# first plot the current over the entire life of the cell
plt.figure(num='current_vs_cycle')
plt.plot(df['Test Time (s)'].values, df['Current (Amps)'].values, 'k-')
plt.xlabel('Test time (s)')
plt.ylabel('Current (Amps)')
plt.tight_layout()

#### Gosh, that's really not helpful
Let's look at cycles 1 through 3 cycles

In [ ]:
sel = (df['Cycle Num'].values > 0)*(df['Cycle Num'].values < 4)

plt.figure(num='current_vs_cycle_try2')
plt.plot(df['Test Time (s)'].values[sel], df['Current (Amps)'].values[sel], 'k-')
plt.xlabel('Test time (s)')
plt.ylabel('Current (Amps)')
plt.tight_layout()

#### What are we looking at here?
* positive current is for charging
* negative current is for discharging
* zero current is a rest

#### On your own, plot the voltage versus time for cycle 1
#### Take a few mintues to experiment with plotting other quantities as well.

#### open up this cell if you need a hint
<font color='white'>
sel = df['Cycle Num'].values == 1
plt.figure(num='voltage_cycle_1')
plt.plot(df['Test Time (s)'].values[sel], df['Voltage'].values[sel], 'k-')
plt.xlabel('Test time (s)')
plt.ylabel('Voltage')
plt.tight_layout()
</font>

#### Let's extract the maximum discharge capacity of the cell throughout its life

In [ ]:
cycs = np.arange(2, ncycmax)
cycs = np.delete(cycs, 10)
Qmaxs = []

plt.figure(num='discharge capacity vs. cycle')
for cyc in cycs:
    sel = (df['Cycle Num'].values == cyc)*(df['Current (Amps)'].values < -0.1)
    t = df['Test Time (s)'].values[sel]
    C = -1*df['Current (Amps)'].values[sel]
    dt = np.diff(t)
    Cmid = (C[1:] + C[:-1])/(2*3600)
    Qmaxs.append(np.sum(dt*Cmid))

    plt.plot(t - t.min(), C, 'k-', alpha=0.01)

plt.xlabel('Test time (s)')
plt.ylabel('Discharge current (Amps)')
plt.tight_layout()
    
plt.figure(num='max capacity vs. cycle')
plt.plot(cycs, Qmaxs, 'k-')
plt.xlabel('Cycle number')
plt.ylabel('Maximum capacity (Amp hrs.)')
plt.tight_layout()

#### Let's extract some features that could help us predict the capacity

Also, let's normalize them and plot them versus cycle

In [ ]:
# featd = {'mean V':[], 'V integral over time':[]}
featd = {'cycles':[], 'mean V':[], 'V integral over time':[]}


# calculate features for each cycles
for cyc in cycs:
    # select the cycle and where the current is negative
    sel = (df['Cycle Num'].values == cyc)*(df['Current (Amps)'].values < -0.1)
    t = df['Test Time (s)'].values[sel]
    C = -1*df['Current (Amps)'].values[sel]
    V = df['Voltage'].values[sel]

    # calculate features
    featd['cycles'].append(cyc)
    featd['mean V'].append(V.mean())
    dt = np.diff(t)
    Vmid = (V[1:] + V[:-1])/2
    featd['V integral over time'].append(np.sum(dt*Vmid))
    
# plot all of the features
features = list(featd.keys())
colors = sns.color_palette()[:len(features)]

plt.figure(num='features vs. cycle')
for feature, color in zip(features, colors):
    feats = featd[feature]
    # normalize the features
    feats = (feats - np.mean(feats))/np.std(feats)
    plt.plot(cycs, feats, marker='', ls='-',
        c=color, alpha=0.9, label=feature)
plt.xlabel('Cycle number')
plt.ylabel('Normalized feature value')
plt.legend()
plt.tight_layout()

#### Before getting into machine learning let's split our dataset into training, validation, and test sets.

In [ ]:
# convert feature dictionary into a Pandas dataframe because it is easier to handle
fdf = pd.DataFrame.from_dict(featd)

# reserve 100 cycles every 100 cycles for the test set
indx_tst = np.arange(fdf.shape[0])
indx_tst = np.mod(np.int32(indx_tst/100) - 1, 2) == 0
indx_trnval = np.invert(indx_tst)

X_ = fdf.values[indx_trnval, :]
y = np.array(Qmaxs)
y_ = y[indx_trnval]
cycs_ = cycs[indx_trnval]
indx = (np.random.random(size=X_.shape[0]) < 0.8)

X_train = X_[indx, :]
y_train = y_[indx]
cycs_train = cycs_[indx]

X_validate = X_[np.invert(indx), :]
y_validate = y_[np.invert(indx)]
cycs_validate = cycs_[np.invert(indx)]

X_test = fdf.values[indx_tst, :]
y_test = y[indx_tst]
cycs_test = cycs[indx_tst]

print('X_train.shape :', X_train.shape)
print('X_validate.shape :', X_validate.shape)
print('X_test.shape :', X_test.shape)

#### Let's evaluate how these features correlate with the SOH
We'll only do so for the training dataset as to no bias our decisions about which features to include

In [ ]:
from scipy.stats import pearsonr

for feature in features:
    feats = np.array(featd[feature])[indx_trnval][indx]
    Qmaxs_ = np.array(Qmaxs)[indx_trnval][indx]

    r, p = pearsonr(feats, Qmaxs_)
    print(feature, 'pearson correlation:', np.round(r, 3), 'and p-value:', np.round(p, 3))

    plt.figure(num=feature + ' correlation')
    plt.plot(Qmaxs_, feats, 'k.')
    plt.plot([np.min(Qmaxs_), np.max(Qmaxs_)], [np.min(feats), np.max(feats)], 'k-')
    plt.xlabel('Maximum capacity (Amp. hrs.)')
    plt.ylabel(feature)
    plt.tight_layout()

#### Let's finally develop a very simple machine learning model

Now let's train a model using linear regression and evaluate the results

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression().fit(X_train, y_train)
p_train = lr.predict(X_train)
p_validate = lr.predict(X_validate)
p_test = lr.predict(X_test)


def cappred(y_train, p_train, y_validate, p_validate,
            y_test=None, p_test=None, name=''):
    r, p = pearsonr(y_train, p_train)
    r, p = pearsonr(y_validate, p_validate)

    r, p = pearsonr(y_train, p_train)
    print('training pearson correlation:', np.round(r, 3), 'and p-value:', np.round(p, 3))
    r, p = pearsonr(y_validate, p_validate)
    print('validation pearson correlation:', np.round(r, 3), 'and p-value:', np.round(p, 3))
    if p_test is not None:
        r, p = pearsonr(y_test, p_test)
        print('testing pearson correlation:', np.round(r, 3), 'and p-value:', np.round(p, 3))

    mae = np.mean(100*np.abs(y_train - p_train)/y_train)
    print('training MAPE:', np.round(mae, 3), '%')
    mae = np.mean(100*np.abs(y_validate - p_validate)/y_validate)
    print('validation MAPE:', np.round(mae, 3), '%')
    if p_test is not None:
        mae = np.mean(100*np.abs(y_test - p_test)/y_test)
        print('testing MAPE:', np.round(mae, 3), '%')

    plt.figure(num=name + ' prediction')
    plt.plot(cycs, y, 'g-', label='SOH')
    plt.plot(cycs_train, p_train, 'k.', alpha=0.5, label='train')
    plt.plot(cycs_validate, p_validate, 'b.', alpha=0.5, label='validate')
    if p_test is not None:
        plt.plot(cycs_test, p_test, 'r.', alpha=0.5, label='test')
    plt.ylim(1.02, 1.08)
    plt.xlabel('Cycle number')
    plt.ylabel('Maximum capacity')
    plt.tight_layout()

    plt.figure(num=name + ' parity', figsize=(4.5, 4))
    plt.plot(y_train, p_train, 'k.', alpha=0.5, label='train')
    plt.plot(y_validate, p_validate, 'b.', alpha=0.5, label='validate')
    if p_test is not None:
        plt.plot(y_test, p_test, 'r.', alpha=0.5, label='test')
    plt.plot([1.02, 1.08], [1.02, 1.08], 'k-')
    plt.xlim(1.02, 1.08)
    plt.ylim(1.02, 1.08)
    plt.xlabel('experiment')
    plt.ylabel('prediction')
    plt.legend()
    plt.tight_layout()
    
cappred(y_train, p_train, y_validate, p_validate, y_test, p_test, 'lr')

Now take 15 or so minutes to experiment with different feature sets and see if you can improve the validation performance. As a hint, look at the voltage and current behavior and see if you can think of features that may correlate well with the SOH.

Next, instead of using an analytical least squares regression approach as before, let's manually define our model and optimize the parameters numerically.

In [ ]:
from scipy.optimize import differential_evolution

def lin_model(X, params):
    p = params[0]*np.ones(X.shape[0])
    for ii in range(X.shape[1]):
        p += params[ii+1]*X[:, ii]
    return p
        
def cost(X, y, params):
    return np.sum((lin_model(X, params) - y)**2)

def cost_opt(params):
    return cost(X_train, y_train, params)

nparam = X_train.shape[1] + 1
params0 = np.zeros((nparam,))
# res = minimize(cost_opt, params0)
bounds = np.zeros((nparam, 2))
bounds[:, 0] = -1
bounds[:, 1] = 1
res = differential_evolution(cost_opt, bounds)
print(res)
params = res.x

p_train = lin_model(X_train, params)
p_validate = lin_model(X_validate, params)
p_test = lin_model(X_test, params)

cappred(y_train, p_train, y_validate, p_validate, y_test, p_test, 'lr-opt')

A potentially more powerful machine learning techniques is Random Forest regression. It is effectively an ensemble of Decision Tree regressors that have been simultaneously trained with randomly generated subsamples of the dataset. We can start exploring the effect of hyperparameters through this exercise.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=2, criterion='mse', min_samples_split=50, min_samples_leaf=25).fit(X_train, y_train)
p_train = rf.predict(X_train)
p_validate = rf.predict(X_validate)
p_test = rf.predict(X_test)

cappred(y_train, p_train, y_validate, p_validate, y_test, p_test, 'rf')

Finally, let's attempt to do a prediction with a neural network model. Here we are using the simplified Keras interface to Tensorflow, a powerful, production-grade software for deep learning applications.

In [ ]:
import tensorflow as tf
from tensorflow import keras

inputs = keras.Input(shape=(X_train.shape[1]), name='f_arr')
x = keras.layers.Dense(4, activation='relu')(inputs)
outputs = keras.layers.Dense(1)(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='dense_pred')

model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001),
              loss='MSE',
              metrics=[])

history = model.fit(X_train, y_train, epochs=500, batch_size=32,
                    validation_data=(X_validate, y_validate))

plt.figure(num='cost vs epoch')
nhist = len(history.history['loss'])
plt.semilogy(range(1, nhist+1), history.history['loss'],
             'b-', label='training loss')
plt.semilogy(range(1, nhist+1), history.history['val_loss'],
             'r-', label='validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.tight_layout()

p_train = np.squeeze(model.predict(X_train))
p_validate = np.squeeze(model.predict(X_validate))
p_test = np.squeeze(model.predict(X_test))             

cappred(y_train, p_train, y_validate, p_validate, y_test, p_test, 'nn')